In [ ]:
import scipy.io
import numpy as np
from sklearn.metrics import auc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
remaining_brainnetome_rois_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/stroke_resilience/patient_data/remaining_brainnetome_rois.mat'
hc_degree_mat_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/atlas_BNA/BNA_240_flipped_N32_retroicor_SBB4_prop_bin_window/HC/Degrees240_binwin_HC.mat'
st_degree_mat_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/atlas_BNA/BNA_240_flipped_N32_retroicor_SBB4_prop_bin_window/ST/Degrees240_binwin.mat'


In [ ]:
minimum_connectivity_threshold = 0.3

In [ ]:
remaining_brainnetome_rois = scipy.io.loadmat(remaining_brainnetome_rois_path)['rois_in_CM_list']

In [ ]:
st_degree_mat = scipy.io.loadmat(st_degree_mat_path)['degrees_bin'][0][0]
st_degree = np.array([np.stack(st_degree_mat[timepoint][0][0]) for timepoint in range(len(st_degree_mat))])

In [ ]:
# use auc only over predefined area of thresholds
def custom_auc(values_over_thresholds):
    # only analyse thresholds above minimum_connectivity_threshold
    minimum_connectivity_threshold_index = int(minimum_connectivity_threshold*10)  - 1 # here thresholds start at bin1
    connectivity_thresholds = np.arange(minimum_connectivity_threshold, 1.1, 0.1)

    return auc(connectivity_thresholds,
                values_over_thresholds[minimum_connectivity_threshold_index:])

In [ ]:
# reduce axis 1 to custom auc
st_degree_auc = np.apply_along_axis(custom_auc, 1, st_degree)

In [ ]:
delta_TP1_TP3 = st_degree_auc[2] - st_degree_auc[0]

In [ ]:
mean_delta_TP1_TP3_per_roi = np.vstack([remaining_brainnetome_rois, np.mean(delta_TP1_TP3, axis=0)]).T

In [ ]:
mean_delta_TP1_TP3_per_roi_df = pd.DataFrame(mean_delta_TP1_TP3_per_roi, columns=['roi', 'mean_delta_TP1_TP3'])

In [ ]:
# plot mean delta_TP1_TP3 value per roi
sns.catplot(x='roi', y='mean_delta_TP1_TP3', data=mean_delta_TP1_TP3_per_roi_df, kind='bar', palette='tab10')

In [ ]:
brainnetome_labels_path = '/Users/jk1/stroke_research/resilience_stroke/longitudinal_analysis/BNA_subregions.xlsx'

In [ ]:
brainnetome_labels_df = pd.read_excel(brainnetome_labels_path)
brainnetome_labels_df['Anatomical and modified Cyto-architectonic descriptions'] = brainnetome_labels_df['Unnamed: 5']
brainnetome_labels_df.drop(columns=['Unnamed: 5'], inplace=True)

In [ ]:
# join labels with delta_TP1_TP3 through roi name
left_annotated_data = pd.merge(mean_delta_TP1_TP3_per_roi_df, brainnetome_labels_df, left_on=['roi'], right_on=['Label ID.L'])
left_annotated_data['hemisphere'] = 'l'

right_annotated_data = pd.merge(mean_delta_TP1_TP3_per_roi_df, brainnetome_labels_df, left_on=['roi'], right_on=['Label ID.R'])
right_annotated_data['hemisphere'] = 'r'

In [ ]:
delta_annotated_data = left_annotated_data.append(right_annotated_data)

In [ ]:
delta_annotated_data.to_csv('delta_degree_tp1_tp3.csv')

In [ ]:
top10_delta = delta_annotated_data.sort_values(by='mean_delta_TP1_TP3', ascending=False).head(10)
top10_delta

In [ ]:
top10_delta.to_csv('top10_delta_degree_tp1_tp3.csv')


In [ ]:
# 3d plot of change in degree_auc
# initialise empty 3d background 70x70x70

delta_cube_atlas = np.zeros((75, 75, 75))
delta_atlas = np.zeros((75, 75, 75))

# populate background with sphere of radius 3 at corresponding mni coordinates
for index, row in delta_annotated_data.iterrows():
    hemisphere = row['hemisphere']
    x, y, z = map(int,row[f'{hemisphere}h.MNI(X,Y,Z)'].split(','))
    delta_cube_atlas[x-3:x+3, y-3:y+3, z-3:z+3] = row['mean_delta_TP1_TP3']
    delta_atlas[x,y,z] = row['mean_delta_TP1_TP3']

In [ ]:
from gsprep.visual_tools.visual import idisplay

# plot delta atlas in 3d
idisplay(delta_atlas)

In [ ]:
import ipyvolume as ipv
ipv.quickvolshow(delta_atlas, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)
